<h1>Table des matières<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-des-librairies" data-toc-modified-id="Import-des-librairies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import des librairies</a></span></li><li><span><a href="#Importation-des-clés" data-toc-modified-id="Importation-des-clés-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Importation des clés</a></span></li><li><span><a href="#Enregistrement-du-modèle" data-toc-modified-id="Enregistrement-du-modèle-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Enregistrement du modèle</a></span></li><li><span><a href="#Déploiment" data-toc-modified-id="Déploiment-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Déploiment</a></span></li><li><span><a href="#Démonstration-d'utilisation-du-modèle" data-toc-modified-id="Démonstration-d'utilisation-du-modèle-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Démonstration d'utilisation du modèle</a></span></li></ul></div>

Ce notebook montre comment déployer un modèle sur le cloud d'Azure et comment l'utiliser pour effectuer des prédictions.

# Import des librairies

https://ml.azure.com/

In [1]:
from azureml.core import Model
from azureml.core import Workspace, Environment
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

import os
import requests
import json
from dotenv import load_dotenv

load_dotenv()

True

source : https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python

# Importation des clés

In [2]:
# On charge dans un premier temps les clés permettant de se connecter
# à l'espace de travail.
tenant_id = os.getenv("YOUR_ML_WORKSPACE_TENANT_ID")
subscription_id = os.getenv("YOUR_ML_WORKSPACE_SUBSCRIPTION_ID")
resource_group = os.getenv("YOUR_ML_WORKSPACE_RESOURCE_GROUP")
workspace_name = os.getenv("YOUR_ML_WORKSPACE_WORKSPACE_NAME")

# Enregistrement du modèle

In [ ]:
def register_model(model_path,
                   model_name,
                   description):

    # On va ensuite être renvoyé vers une page d'identification
    # pour valider l'authentification.
    ia = InteractiveLoginAuthentication(tenant_id=tenant_id)
    
    # Instantiation de l'espace de travail
    ws = Workspace.get(name=workspace_name,
                       subscription_id=subscription_id,
                       resource_group=resource_group,
                       auth=ia)

    # Enregistrement du modèle sur le cloud
    model = Model.register(workspace=ws,
                           model_path=model_path,
                           model_name=model_name,
                           description=description)
    return model

In [4]:
model_path = 'data/clean'
model_name = 'lstm_glove_hyperopt'
description = 'Modèle avancé - Analyse Sentiment - LSTM + GloVe'

model = register_model(model_path, model_name, description)

Registering model lstm_glove_hyperopt


# Déploiment

In [ ]:
def deploy_model(env_name: str, packages: list):

    # On installe toutes librairies requises au fonctionnement de la pipeline
    myenv = Environment(env_name)

    for package in packages:
        myenv.python.conda_dependencies.add_pip_package(package)

    # Script de la pipeline permettant de retourner les scores de prédiction
    inference_config = InferenceConfig(environment=myenv,
                                       entry_script='score.py')

    # Configuration de la machine virtuelle
    deployment_config = AciWebservice.deploy_configuration(cpu_cores=0.5,
                                                           memory_gb=1)

    # Déploiement du modèle
    service = Model.deploy(workspace=ws,
                           name='tweet-sentiment-service',
                           models=[model],
                           inference_config=inference_config,
                           deployment_config=deployment_config)

    service.wait_for_deployment(show_output=True)

    # Verbose
    print(service.state)
    
    return service

In [8]:
packages = [
    "pickle5",
    "tensorflow",
    "azureml-core",
    "contractions",
    "nltk"]

service = deploy_model("modelavance", packages)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-05-15 11:04:52+02:00 Creating Container Registry if not exists.
2022-05-15 11:04:52+02:00 Registering the environment.
2022-05-15 11:04:55+02:00 Building image..
2022-05-15 11:10:24+02:00 Generating deployment configuration..
2022-05-15 11:10:27+02:00 Submitting deployment to compute..
2022-05-15 11:10:32+02:00 Checking the status of deployment tweet-sentiment-service..
2022-05-15 11:12:41+02:00 Checking the status of inference endpoint tweet-sentiment-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [ ]:
# print(service.get_logs())

# Démonstration d'utilisation du modèle

In [12]:
rest_endpoint = 'http://c66cd70a-ae16-4871-a864-caa59264ead0.westeurope.azurecontainer.io/score'
headers = {'Content-Type': 'application/json'}

# Provide a text example
data = json.dumps({'text': 'She was in love'})

# Call with POST request
response = requests.post(rest_endpoint, data=data, headers=headers)

# Print result
print('Status : ', response.status_code)
print('Tweet : ', (json.loads(data)).get('text'))
print('Label : ', (response.json()).get('Label'))
print('Score : ', (response.json()).get('Score'))
print('Temps : ', (response.json()).get('Temps'))

Status :  200
Tweet :  She was in love
Label :  Positif
Score :  0.9991021156311035
Temps :  0.08948421478271484


In [13]:
service.delete()
model.delete()

No service with name tweet-sentiment-service found to delete.
